In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import ollama

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. Respond to greetings and general conversation politely."
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "When a user asks for information that requires external data or action, use the available tools to get that information Specifically"

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(model="llama3.2", messages=messages)
    return response['message']['content']

gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
#tools = [{"type": "function", "function": price_function}]
tools = [
    {
        "type":"function",
        "function":{
            "name": "get_ticket_price",
            "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
             "parameters": {
                 "type": "object",
                 "properties": {
                     "destination_city": {
                         "type": "string",
                         "description": "The city that the customer wants to travel to"
                         },
                    },
                    "required": ["destination_city"],
                    "additionalProperties": False
                },
            },
    }
]

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = tool_call.function.arguments
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
       # "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(model="llama3.2", messages=messages,tools=tools)
    if response['message'].get('tool_calls'):
        message = response['message']
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = ollama.chat(model="llama3.2", messages=messages)
    
    return response['message']['content']

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()